In [1]:
from transformers import AutoModelForCausalLM
import torch.distributed._shard.checkpoint as dist_cp
import torch

# source: https://discuss.huggingface.co/t/transformers-trainer-accelerate-fsdp-how-do-i-load-my-model-from-a-checkpoint/61585

model = AutoModelForCausalLM.from_pretrained(
    "/data/nvme3n1p1/adal_workspace/models/LLama-3.1-KazLLM-1.0-8B-Frank-UPDTok/orig_model",
    torch_dtype=torch.bfloat16,
    device_map="cpu",
)

/root/miniconda3/envs/modern_tune/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_2592365/4010403221.py:2: DeprecationWarning: `torch.distributed._shard.checkpoint` will be deprecated, use `torch.distributed.checkpoint` instead
  import torch.distributed._shard.checkpoint as dist_cp
Loading checkpoint shards: 100%|██████████| 7/7 [00:02<00:00,  2.95it/s]


In [2]:
weights_200 = torch.load("/data/nvme3n1p1/adal_workspace/llm_train/models/franky_model_pt/model_18-02-24_2000.pt", map_location="cpu")

/tmp/ipykernel_2592365/2118760279.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights_200 = torch.load("/data/nvme3n1p1/adal_workspace/llm_train/models/franky_model_

In [3]:
weights_200['model.layers.32.post_attention_layernorm.weight']

tensor([0.5117, 0.4785, 0.5078,  ..., 0.5117, 0.4707, 0.4121],
       dtype=torch.bfloat16)

In [ ]:
for param in weights_200:
    if (weights_200[f'{param}'] != model.state_dict()[f'{param}']).any():
        print(param)

model.embed_tokens.weight
model.layers.0.self_attn.q_proj.weight
model.layers.0.self_attn.k_proj.weight
model.layers.0.self_attn.v_proj.weight
model.layers.0.self_attn.o_proj.weight
model.layers.0.input_layernorm.weight


model.layers.33.self_attn.q_proj.weight
model.layers.33.self_attn.k_proj.weight
model.layers.33.self_attn.v_proj.weight
model.layers.33.self_attn.o_proj.weight
model.layers.33.input_layernorm.weight
lm_head.weight


In [15]:
for param in weights_200:
    print(param)

model.embed_tokens.weight
model.layers.0.self_attn.q_proj.weight
model.layers.0.self_attn.k_proj.weight
model.layers.0.self_attn.v_proj.weight
model.layers.0.self_attn.o_proj.weight
model.layers.0.mlp.gate_proj.weight
model.layers.0.mlp.up_proj.weight
model.layers.0.mlp.down_proj.weight
model.layers.0.input_layernorm.weight
model.layers.0.post_attention_layernorm.weight
model.layers.1.self_attn.q_proj.weight
model.layers.1.self_attn.k_proj.weight
model.layers.1.self_attn.v_proj.weight
model.layers.1.self_attn.o_proj.weight
model.layers.1.mlp.gate_proj.weight
model.layers.1.mlp.up_proj.weight
model.layers.1.mlp.down_proj.weight
model.layers.1.input_layernorm.weight
model.layers.1.post_attention_layernorm.weight
model.layers.2.self_attn.q_proj.weight
model.layers.2.self_attn.k_proj.weight
model.layers.2.self_attn.v_proj.weight
model.layers.2.self_attn.o_proj.weight
model.layers.2.mlp.gate_proj.weight
model.layers.2.mlp.up_proj.weight
model.layers.2.mlp.down_proj.weight
model.layers.2.inp

In [19]:
(weights_200['model.layers.0.self_attn.q_proj.weight'] != model.state_dict()['model.layers.0.self_attn.q_proj.weight']).any()

tensor(True)

In [4]:
model.state_dict()['model.layers.32.post_attention_layernorm.weight']

tensor([0.5117, 0.4785, 0.5078,  ..., 0.5117, 0.4707, 0.4121],
       dtype=torch.bfloat16)

In [9]:
model.load_state_dict(weights_200, strict=False)

<All keys matched successfully>

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("/data/nvme3n1p1/adal_workspace/models/LLama-3.1-KazLLM-1.0-8B-Frank-UPDTok/orig_model")

In [6]:
import os
path = "/data/nvme3n1p1/adal_workspace/llm_train/models/franky_model_hf-2000"
if not os.path.exists(path):
    os.makedirs(path)

model.save_pretrained(path)
tokenizer.save_pretrained(path) 

('/data/nvme3n1p1/adal_workspace/llm_train/models/franky_model_hf-2000/tokenizer_config.json',
 '/data/nvme3n1p1/adal_workspace/llm_train/models/franky_model_hf-2000/special_tokens_map.json',
 '/data/nvme3n1p1/adal_workspace/llm_train/models/franky_model_hf-2000/tokenizer.json')

In [24]:
model.save_pretrained("/data/nvme3n1p1/adal_workspace/models/LLama-3.1-KazLLM-1.0-8B-Frank-UPDTok/outputs/folder-model_17-02-24_2000")
tokenizer.save_pretrained("/data/nvme3n1p1/adal_workspace/models/LLama-3.1-KazLLM-1.0-8B-Frank-UPDTok/outputs/folder-model_17-02-24_2000")

('/data/nvme3n1p1/adal_workspace/models/LLama-3.1-KazLLM-1.0-8B-Frank-UPDTok/outputs/folder-model_17-02-24_2000/tokenizer_config.json',
 '/data/nvme3n1p1/adal_workspace/models/LLama-3.1-KazLLM-1.0-8B-Frank-UPDTok/outputs/folder-model_17-02-24_2000/special_tokens_map.json',
 '/data/nvme3n1p1/adal_workspace/models/LLama-3.1-KazLLM-1.0-8B-Frank-UPDTok/outputs/folder-model_17-02-24_2000/tokenizer.json')

In [11]:
model_another = AutoModelForCausalLM.from_pretrained(
    "/data/nvme3n1p1/adal_workspace/models/LLama-3.1-KazLLM-1.0-8B-Frank-UPDTok/orig_model",
    torch_dtype=torch.bfloat16,
    device_map="cpu",
)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 7/7 [00:03<00:00,  2.18it/s]


In [14]:
model_another.state_dict()['model.layers.32.post_attention_layernorm.weight']

tensor([0.5117, 0.4785, 0.5078,  ..., 0.5117, 0.4707, 0.4121],
       dtype=torch.bfloat16)

In [17]:
model_new

<All keys matched successfully>

In [5]:
weights_200

OrderedDict([('model.embed_tokens.weight',
              tensor([[-2.6611e-02,  8.1177e-03, -7.5684e-03,  ...,  1.9836e-03,
                        1.5869e-02, -1.9836e-03],
                      [-7.1716e-03, -8.0872e-04, -1.2756e-02,  ...,  1.6357e-02,
                       -9.8267e-03, -1.3367e-02],
                      [ 8.5449e-03, -2.7344e-02, -4.5776e-04,  ...,  5.4016e-03,
                        1.6251e-03,  1.4893e-02],
                      ...,
                      [ 2.2127e-23,  3.9033e-24,  2.1610e-23,  ...,  6.3693e-23,
                       -2.6496e-24, -2.3575e-23],
                      [ 2.2851e-23, -2.2101e-24, -2.2230e-23,  ...,  2.7917e-23,
                        8.6854e-24, -3.7016e-23],
                      [-8.8508e-23, -7.5687e-23,  6.4882e-24,  ...,  5.8937e-24,
                       -6.4520e-23, -2.7142e-24]], dtype=torch.bfloat16)),
             ('model.layers.0.self_attn.q_proj.weight',
              tensor([[ 0.0098, -0.0175, -0.0125,  ...,  0.0015

In [3]:
state_dict = {
        "model": model.state_dict()
    }

dist_cp.load_state_dict(
                state_dict=state_dict,
                storage_reader= dist_cp.FileSystemReader("/data/nvme3n1p1/adal_workspace/models/LLama-3.1-KazLLM-1.0-8B-Frank-UPDTok/outputs/model_17-02-24_2000.pt"),
                no_dist=True,
            )

/tmp/ipykernel_3468401/1563511204.py:5: FutureWarning: `load_state_dict` is deprecated and will be removed in future versions. Please use `load` instead.
  dist_cp.load_state_dict(


CheckpointException: CheckpointException ranks:dict_keys([0])
Traceback (most recent call last): (RANK 0)
  File "/root/miniconda3/envs/modern_tune/lib/python3.12/site-packages/torch/distributed/checkpoint/utils.py", line 164, in reduce_scatter
    local_data = map_fun()
                 ^^^^^^^^^
  File "/root/miniconda3/envs/modern_tune/lib/python3.12/site-packages/torch/distributed/checkpoint/logger.py", line 83, in wrapper
    result = func(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^
  File "/root/miniconda3/envs/modern_tune/lib/python3.12/site-packages/torch/distributed/checkpoint/state_dict_loader.py", line 207, in local_step
    metadata = storage_reader.read_metadata()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/miniconda3/envs/modern_tune/lib/python3.12/site-packages/torch/distributed/checkpoint/filesystem.py", line 679, in read_metadata
    with self.fs.create_stream(path, "rb") as metadata_file:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root

In [ ]:
model.load_state_dict(state_dict["model"])

new_state_dict = model.state_dict()

In [8]:
from datasets import load_dataset

In [9]:
ds = load_dataset("json", data_files="/data/nvme5n1p1/sft_ds/kk/2024.09.23_instruct_kk_QnA-hakurei-open-instruct-v1-train_14638854.json")

Generating train split: 199106 examples [00:01, 187493.17 examples/s]


In [13]:
ds['train'][12000]

{'instruction': ' Рецепт жасаңыз, содан кейін оны жасауға қажетті ингредиенттерді жазыңыз. Рецепт: Құрамы: - 2 кесе ұн - 3 жұмыртқа - 1 стакан қант - 1 шай қасық қопсытқыш - 1/2 шай қасық тұз',
 'input': '',
 'output': ' 1. Ыдысқа ұн, қопсытқыш және тұзды араластырыңыз.'}

In [ ]:
from huggingface_hub import HfApi, login

login(token="", add_to_git_credential=False)

api = HfApi()

api.create_repo(repo_id="AdalAbilbekov/small_lm_test1", repo_type="model", exist_ok=True, private=True)

# api.upload_file(
#     path_in_repo="model_config.json",
#     path_or_fileobj="/scratch/adal_abilbekov/models/Llama_1.5B_10-02-2025-V1/209000/model_config.json",
#     repo_id="AdalAbilbekov/small_lmV1_209000",
# )

api.upload_folder(
    folder_path="/data/nvme3n1p1/adal_workspace/small_lm/sft_model_18-02-24_13000-HF",
    repo_id="AdalAbilbekov/small_lm_test1",
    repo_type="model"
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /data/nvme2n1p1/token
Login successful


model.safetensors: 100%|██████████| 2.77G/2.77G [00:52<00:00, 53.2MB/s]


CommitInfo(commit_url='https://huggingface.co/AdalAbilbekov/small_lm_test1/commit/c26cada0017eb28fe30ef0a6846e4852d51f626d', commit_message='Upload folder using huggingface_hub', commit_description='', oid='c26cada0017eb28fe30ef0a6846e4852d51f626d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/AdalAbilbekov/small_lm_test1', endpoint='https://huggingface.co', repo_type='model', repo_id='AdalAbilbekov/small_lm_test1'), pr_revision=None, pr_num=None)